<a href="https://colab.research.google.com/github/camillefougere/PredictivePolicingCaseStudy/blob/main/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import useful packages:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
from IPython.display import display
import datetime as dt

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


**Import dataset:** data is Los Angeles crime data rom 2010-2019. The dataset can be found on Los Angeles Open Data website, at https://data.lacity.org/Public-Safety/Crime-Data-from-2010-to-2019/63jg-8b9z

**A writeup** of the dataset's relevance to this project and it's methodology can be found at https://docs.google.com/document/d/1pi6F1nr2BcM4H9yEOqhl2t7LhEff7BFpkPl-IcwNyNY/edit?usp=sharing



In [ ]:
total_crime = pd.read_csv('/gdrive/MyDrive/capstone/Crime_Data_from_2010_to_2019.csv')

In [ ]:
total_crime['Date Rptd'] = pd.to_datetime(total_crime['Date Rptd'])
total_crime['DATE OCC'] = pd.to_datetime(total_crime['DATE OCC'])
display(total_crime.head(5))
# limit dataset to only crimes that were reported/occurred in 2012
dates_2012 = (total_crime['DATE OCC'] >= '2012-1-1') & (total_crime['DATE OCC'] < '2013-1-1')
df_2012 = total_crime.loc[dates_2012]
df_2012.sort_values(by='DATE OCC', inplace=True)
# check that the first possible date is 1/1/2012 and the last possible is 12/31/2012
print('First Possible Date: ' + str(min(list(df['DATE OCC']))))
print('Last Possible Date: ' + str(max(list(df['DATE OCC']))))
display(df_2012.head(5))
# export as a csv so don't need to upload much larger file
df_2012.to_csv('Crime_Data_During_2012.csv')

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,1307355,2010-02-20,2010-02-20,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,0913 1814 2000,48,M,H,501.0,SINGLE FAMILY DWELLING,NaN,NaN,AA,Adult Arrest,900.0,NaN,NaN,NaN,300 E GAGE AV,NaN,33.9825,-118.2695
1,11401303,2010-09-13,2010-09-12,45,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329,0,M,W,101.0,STREET,NaN,NaN,IC,Invest Cont,740.0,NaN,NaN,NaN,SEPULVEDA BL,MANCHESTER AV,33.9599,-118.3962
2,70309629,2010-08-09,2010-08-09,1515,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,0344,0,M,H,103.0,ALLEY,NaN,NaN,IC,Invest Cont,946.0,NaN,NaN,NaN,1300 E 21ST ST,NaN,34.0224,-118.2524
3,90631215,2010-01-05,2010-01-05,150,6,Hollywood,646,2,900,VIOLATION OF COURT ORDER,1100 0400 1402,47,F,W,101.0,STREET,102.0,HAND GUN,IC,Invest Cont,900.0,998.0,NaN,NaN,CAHUENGA BL,HOLLYWOOD BL,34.1016,-118.3295
4,100100501,2010-01-03,2010-01-02,2100,1,Central,176,1,122,"RAPE, ATTEMPTED",0400,47,F,H,103.0,ALLEY,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,122.0,NaN,NaN,NaN,8TH ST,SAN PEDRO ST,34.0387,-118.2488


First Possible Date: 2012-01-01
Last Possible Date: 2012-12-31


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
513378,130508806,2013-04-21,2012-01-01,1,5,Harbor,522,2,354,THEFT OF IDENTITY,1822 1202,59,F,H,501.0,SINGLE FAMILY DWELLING,NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,600 GULF AV,NaN,33.7765,-118.2726
447563,151920788,2015-10-27,2012-01-01,1,19,Mission,1934,2,354,THEFT OF IDENTITY,1822 1218,24,M,B,726.0,POLICE FACILITY,NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,11100 N SEPULVEDA BL,NaN,34.2721,-118.4675
466730,121706321,2012-02-13,2012-01-01,1200,17,Devonshire,1798,2,354,THEFT OF IDENTITY,0377 0928 1822,46,F,W,501.0,SINGLE FAMILY DWELLING,NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,15800 BAHAMA ST,NaN,34.2329,-118.4775
411430,170816763,2017-10-10,2012-01-01,800,8,West LA,889,2,354,THEFT OF IDENTITY,1822,56,M,H,502.0,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,2000 S GARTH AV,NaN,34.0408,-118.3791
488069,141215190,2014-06-27,2012-01-01,1200,12,77th Street,1258,2,813,CHILD ANNOYING (17YRS & UNDER),1251 1259 1820 0319 0522,14,F,H,501.0,SINGLE FAMILY DWELLING,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,813.0,NaN,NaN,NaN,73RD ST,MAIN ST,33.9739,-118.2739


In [ ]:
# repload data limited to 2012:
df = pd.read_csv('/gdrive/MyDrive/capstone/Crime_Data_During_2012.csv')
display(df.head(5))

,Unnamed: 0,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,513378,130508806,2013-04-21,2012-01-01,1,5,Harbor,522,2,354,THEFT OF IDENTITY,1822 1202,59,F,H,501.0,SINGLE FAMILY DWELLING,NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,600 GULF AV,NaN,33.7765,-118.2726
1,447563,151920788,2015-10-27,2012-01-01,1,19,Mission,1934,2,354,THEFT OF IDENTITY,1822 1218,24,M,B,726.0,POLICE FACILITY,NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,11100 N SEPULVEDA BL,NaN,34.2721,-118.4675
2,466730,121706321,2012-02-13,2012-01-01,1200,17,Devonshire,1798,2,354,THEFT OF IDENTITY,0377 0928 1822,46,F,W,501.0,SINGLE FAMILY DWELLING,NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,15800 BAHAMA ST,NaN,34.2329,-118.4775
3,411430,170816763,2017-10-10,2012-01-01,800,8,West LA,889,2,354,THEFT OF IDENTITY,1822,56,M,H,502.0,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,2000 S GARTH AV,NaN,34.0408,-118.3791
4,488069,141215190,2014-06-27,2012-01-01,1200,12,77th Street,1258,2,813,CHILD ANNOYING (17YRS & UNDER),1251 1259 1820 0319 0522,14,F,H,501.0,SINGLE FAMILY DWELLING,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,813.0,NaN,NaN,NaN,73RD ST,MAIN ST,33.9739,-118.2739


In [ ]:
# clean and explore the dataset by variable:

# Unnamed: 0 is a leftover column from limiting to 2012 representing the index in the original df,
# we don't need this
if 'Unnamed: 0' in df.columns:
  df.drop(columns='Unnamed: 0', inplace=True)
# DR_NO is a variable representing Division of Records Number: Official file number made up of a 2 digit
# year, area ID, and 5 digits
# Not especially relevant to this project

# Date Rptd: represents the date on which the crime was reported
df['Date Rptd'] = pd.to_datetime(df['Date Rptd'])
print('DATE REPORTED: ')
print('_________________________________________________________________________')
print(df['Date Rptd'].describe(), '\n')

DATE REPORTED: 
_________________________________________________________________________
count                  201429
unique                   1983
top       2012-08-20 00:00:00
freq                      686
first     2012-01-01 00:00:00
last      2020-11-22 00:00:00
Name: Date Rptd, dtype: object 



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  from ipykernel import kernelapp as app


In [ ]:

# DATE OCC: represents the date on which the crime occurred
print('DATE OCC: ')
print('_________________________________________________________________________')
print(df['DATE OCC'].describe(), '\n')

DATE OCC: 
_________________________________________________________________________
count         201429
unique           366
top       2012-01-01
freq            1811
Name: DATE OCC, dtype: object 



In [ ]:
# TIME OCC: represents the time during which the crime occurred (in 24 hour military time)
# will need to cast this to datetime time

dates_occ = []
for i in range(len(df)):
  if len(str(df.loc[i]['TIME OCC'])) == 4:
    fixed_time = str(df.loc[i]['TIME OCC'])[:2] + ':' + str(df.loc[i]['TIME OCC'])[-2:]
  if len(str(df.loc[i]['TIME OCC'])) == 3:
    fixed_time = '0' + str(df.loc[i]['TIME OCC'])[:1] + ':' + str(df.loc[i]['TIME OCC'])[-2:]
  if len(str(df.loc[i]['TIME OCC'])) == 2:
    fixed_time = '00' + ':' + str(df.loc[i]['TIME OCC'])[-2:]
  if len(str(df.loc[i]['TIME OCC'])) == 1:
    fixed_time = '00:0' + str(df.loc[i]['TIME OCC'])[-1:]
  combined = str(df.loc[i]['DATE OCC']) + ' ' + fixed_time
  date_time = dt.datetime.strptime(combined, '%Y-%m-%d %H:%M')
  dates_occ.append(date_time)

df.insert(4, 'Date Occd', dates_occ)
display(df.head(5))
print('Date Occd: ')
print('_________________________________________________________________________')
print(df['Date Occd'].describe(), '\n')

,DR_NO,Date Rptd,DATE OCC,TIME OCC,Date Occd,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,130508806,2013-04-21,2012-01-01,1,2012-01-01 00:01:00,5,Harbor,522,2,354,THEFT OF IDENTITY,1822 1202,59,F,H,501.0,SINGLE FAMILY DWELLING,NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,600 GULF AV,NaN,33.7765,-118.2726
1,151920788,2015-10-27,2012-01-01,1,2012-01-01 00:01:00,19,Mission,1934,2,354,THEFT OF IDENTITY,1822 1218,24,M,B,726.0,POLICE FACILITY,NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,11100 N SEPULVEDA BL,NaN,34.2721,-118.4675
2,121706321,2012-02-13,2012-01-01,1200,2012-01-01 12:00:00,17,Devonshire,1798,2,354,THEFT OF IDENTITY,0377 0928 1822,46,F,W,501.0,SINGLE FAMILY DWELLING,NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,15800 BAHAMA ST,NaN,34.2329,-118.4775
3,170816763,2017-10-10,2012-01-01,800,2012-01-01 08:00:00,8,West LA,889,2,354,THEFT OF IDENTITY,1822,56,M,H,502.0,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,2000 S GARTH AV,NaN,34.0408,-118.3791
4,141215190,2014-06-27,2012-01-01,1200,2012-01-01 12:00:00,12,77th Street,1258,2,813,CHILD ANNOYING (17YRS & UNDER),1251 1259 1820 0319 0522,14,F,H,501.0,SINGLE FAMILY DWELLING,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,813.0,NaN,NaN,NaN,73RD ST,MAIN ST,33.9739,-118.2739


Date Occd: 
_________________________________________________________________________
count                  201429
unique                  58020
top       2012-01-01 12:00:00
freq                      455
first     2012-01-01 00:01:00
last      2012-12-31 23:59:00
Name: Date Occd, dtype: object 



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.


In [ ]:
# AREA: The LAPD has 21 Community Police Stations referred to as Geographic Areas within the department.
# These Geographic Areas are sequentially numbered from 1-21.

# column name is 'AREA ', need to rename to 'AREA'
df.rename(columns={'AREA ':'AREA'}, inplace=True)

print('AREA: ')
print('_________________________________________________________________________')
print(df['AREA'].value_counts(), '\n')

# AREA NAME: The 21 Geographic Areas or Patrol Divisions are also given a name designation that references
# a landmark or the surrounding community that it is responsible for. For example 77th Street Division is
# located at the intersection of South Broadway and 77th Street, serving neighborhoods in South Los Angeles.
print('AREA NAME: ')
print('_________________________________________________________________________')
print(df['AREA NAME'].value_counts(), '\n')

# Rpt Dist No: A four-digit code that represents a sub-area within a Geographic Area. All crime records
# reference the "RD" that it occurred in for statistical comparisons. Find LAPD Reporting Districts on the
# LA City GeoHub at http://geohub.lacity.org/datasets/c4f83909b81d4786aa8ba8a74a4b4db1_4
print('Rpt Dist No: ')
print('_________________________________________________________________________')
print(df['Rpt Dist No'].value_counts())

AREA: 
_________________________________________________________________________
12    14283
3     13108
15    11276
18    10588
19    10584
14    10306
11     9843
9      9648
17     9427
5      9421
6      9313
10     9022
21     8921
13     8881
2      8600
8      8577
20     8518
1      8073
7      8038
16     7934
4      7068
Name: AREA, dtype: int64 

AREA NAME: 
_________________________________________________________________________
77th Street    14283
Southwest      13108
N Hollywood    11276
Southeast      10588
Mission        10584
Pacific        10306
Northeast       9843
Van Nuys        9648
Devonshire      9427
Harbor          9421
Hollywood       9313
West Valley     9022
Topanga         8921
Newton          8881
Rampart         8600
West LA         8577
Olympic         8518
Central         8073
Wilshire        8038
Foothill        7934
Hollenbeck      7068
Name: AREA NAME, dtype: int64 

Rpt Dist No: 
___________________________________________________________________

In [ ]:
# Part 1-2: refers to the type of crime that occurred. Part 1 crimes are violent and property crimes,
# such as aggravated assault, forcible rape, murder, and robbery (classified as violent) and arson,
# burglary, larceny-theft, and motor vehicle theft (classified as property crimes). Part 2 crimes are simple
# assault, curfew offenses and loitering, embezzlement, forgery and counterfeiting, disorderly conduct,
# driving under the influence, drug offenses, fraud, gambling, liquor offenses, offenses against the family,
# prostitution, public drunkenness, runaways, sex offenses, stolen property, vandalism, vagrancy, and weapons offenses.
print('Part 1-2: ')
print('_________________________________________________________________________')
print(df['Part 1-2'].value_counts())

Part 1-2: 
_________________________________________________________________________
1    107674
2     93755
Name: Part 1-2, dtype: int64


In [ ]:
# Crm Cd: Indicates the crime committed. (Same as Crime Code 1)
print('CRIME CODE: ')
print('_________________________________________________________________________')
print(df['Crm Cd'].value_counts(), '\n')

# Crm Cd Desc: Defines the Crime Code provided.
print('CRIME CODE DESCRIPTION: ')
print('_________________________________________________________________________')
print(df['Crm Cd Desc'].value_counts(), '\n')

# Mocodes: Modus Operandi: Activities associated with the suspect in commission of the crime.
# See attached PDF for list of MO Codes in numerical order.
# https://data.lacity.org/api/views/y8tr-7khq/files/3a967fbd-f210-4857-bc52-60230efe256c?download=true&filename=MO%20CODES%20(numerical%20order).pdf
print('MODUS OPERANDI CODES: ')
print('_________________________________________________________________________')
print(df['Mocodes'].value_counts())

CRIME CODE: 
_________________________________________________________________________
624    19824
440    16105
330    15916
310    15117
510    14826
       ...  
924        1
475        1
349        1
353        1
436        1
Name: Crm Cd, Length: 123, dtype: int64 

CRIME CODE DESCRIPTION: 
_________________________________________________________________________
BATTERY - SIMPLE ASSAULT                                    19824
THEFT PLAIN - PETTY ($950 & UNDER)                          16105
BURGLARY FROM VEHICLE                                       15916
BURGLARY                                                    15117
VEHICLE - STOLEN                                            14826
                                                            ...  
DRUNK ROLL                                                      1
THEFT, COIN MACHINE - ATTEMPT                                   1
BEASTIALITY, CRIME AGAINST NATURE SEXUAL ASSLT WITH ANIM        1
GRAND THEFT / AUTO REPAIR          

In [ ]:
# Vict Age: represents the age of the victim
# some ages appear to be <0, replace with NaN
df.loc[df['Vict Age'] < 0, 'Vict Age'] = np.nan

print('VICTIM AGE: ')
print('_________________________________________________________________________')
print(df['Vict Age'].describe(), '\n')

# Vict Sex: represents the sex of the victim, F - Female M - Male X - Unknown
# some sex is represented as H, no clear mapping for this, so change these rows to X
df.loc[df['Vict Sex'] == 'H', 'Vict Sex'] = 'X'

print('VICTIM SEX: ')
print('_________________________________________________________________________')
print(df['Vict Sex'].value_counts(), '\n')

# Vict Descent: Descent Code: A - Other Asian B - Black C - Chinese D - Cambodian F - Filipino
# G - Guamanian H - Hispanic/Latin/Mexican I - American Indian/Alaskan Native J - Japanese K - Korean
# L - Laotian O - Other P - Pacific Islander S - Samoan U - Hawaiian V - Vietnamese W - White X - Unknown Z - Asian Indian
# some descent represented as '-', swith this to Other
df.loc[df['Vict Descent'] == '-', 'Vict Descent'] = 'O'

print('VICTIM DESCENT: ')
print('_________________________________________________________________________')
print(df['Vict Descent'].value_counts())

VICTIM AGE: 
_________________________________________________________________________
count    201352.000000
mean         31.676715
std          20.244881
min           0.000000
25%          20.000000
50%          31.000000
75%          46.000000
max          99.000000
Name: Vict Age, dtype: float64 

VICTIM SEX: 
_________________________________________________________________________
M    94507
F    87961
X     1026
Name: Vict Sex, dtype: int64 

VICTIM DESCENT: 
_________________________________________________________________________
H    70138
W    51767
B    33508
O    19007
A     4448
X     3199
K      918
F      269
I       71
C       60
P       28
J       25
U       25
V       12
G        5
Z        5
S        4
D        1
Name: Vict Descent, dtype: int64


In [ ]:
# Premis Cd: The type of structure, vehicle, or location where the crime took place.
print('PREMISE CODE: ')
print('_________________________________________________________________________')
print(df['Premis Cd'].value_counts(), '\n')

# Premis Desc: Defines the Premise Code provided.
print('PREMISE DESCRIPTION: ')
print('_________________________________________________________________________')
print(df['Premis Desc'].value_counts(), '\n')

# Weapon Used Cd: The type of weapon used in the crime.
print('WEAPON USED CODE: ')
print('_________________________________________________________________________')
print(df['Weapon Used Cd'].value_counts(), '\n')

# Weapon Desc: Defines the Weapon Used Code provided.
print('WEAPON DESCRIPTION: ')
print('_________________________________________________________________________')
print(df['Weapon Desc'].value_counts(), '\n')


PREMISE CODE: 
_________________________________________________________________________
501.0    44064
101.0    43438
502.0    27783
108.0    13957
102.0    10271
         ...  
132.0        1
139.0        1
734.0        1
746.0        1
808.0        1
Name: Premis Cd, Length: 197, dtype: int64 

PREMISE DESCRIPTION: 
_________________________________________________________________________
SINGLE FAMILY DWELLING                                44064
STREET                                                43438
MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)          27783
PARKING LOT                                           13957
SIDEWALK                                              10271
                                                      ...  
ORANGE LINE PARKING LOT                                   1
HOCKEY RINK/ICE HOCKEY                                    1
CULTURAL SIGNIFICANCE/MONUMENT                            1
AUTO DEALERSHIP (CHEVY, FORD, BMW, MERCEDES, ETC.)        1
AMTRA

In [ ]:
# Status: Status of the case. (IC is the default)
print('CASE STATUS: ')
print('_________________________________________________________________________')
print(df['Status'].value_counts(), '\n')

# Status Desc: Defines the Status Code provided.
print('CASE STATUS DESC: ')
print('_________________________________________________________________________')
print(df['Status Desc'].value_counts(), '\n')

CASE STATUS: 
_________________________________________________________________________
IC    151665
AO     24779
AA     22600
JA      1812
JO       570
CC         2
TH         1
Name: Status, dtype: int64 

CASE STATUS DESC: 
_________________________________________________________________________
Invest Cont     151665
Adult Other      24779
Adult Arrest     22600
Juv Arrest        1812
Juv Other          570
UNK                  3
Name: Status Desc, dtype: int64 



In [ ]:
# Crm Cd 1: Indicates the crime committed. Crime Code 1 is the primary and most serious one.
# Crime Code 2, 3, and 4 are respectively less serious offenses. Lower crime class numbers are more serious.
print('CRIME CODE #1: ')
print('_________________________________________________________________________')
print(df['Crm Cd 1'].value_counts(), '\n')

# Crm Cd 2: May contain a code for an additional crime, less serious than Crime Code 1.
print('CRIME CODE #2: ')
print('_________________________________________________________________________')
print(df['Crm Cd 2'].value_counts(), '\n')

# Crm Cd 3: May contain a code for an additional crime, less serious than Crime Code 1.
print('CRIME CODE #3: ')
print('_________________________________________________________________________')
print(df['Crm Cd 3'].value_counts(), '\n')

# Crm Cd 4: May contain a code for an additional crime, less serious than Crime Code 1.
print('CRIME CODE #4: ')
print('_________________________________________________________________________')
print(df['Crm Cd 4'].value_counts(), '\n')

CRIME CODE #1: 
_________________________________________________________________________
624.0    19835
440.0    16098
330.0    15934
310.0    15117
510.0    14826
         ...  
840.0        1
349.0        1
521.0        1
353.0        1
475.0        1
Name: Crm Cd 1, Length: 124, dtype: int64 

CRIME CODE #2: 
_________________________________________________________________________
998.0    10340
860.0      151
740.0      112
745.0      109
910.0      108
         ...  
439.0        1
954.0        1
235.0        1
627.0        1
625.0        1
Name: Crm Cd 2, Length: 86, dtype: int64 

CRIME CODE #3: 
_________________________________________________________________________
998.0    166
999.0      3
910.0      3
821.0      3
745.0      2
946.0      2
930.0      2
626.0      2
860.0      2
510.0      1
653.0      1
649.0      1
820.0      1
812.0      1
521.0      1
Name: Crm Cd 3, dtype: int64 

CRIME CODE #4: 
_______________________________________________________________________

In [ ]:
df['Cross Street'] = df['Cross Street'].astype('object')
df['Cross Street'].fillna('Missing', inplace=True)
# # lots of inconsistent spacing in location and cross st strings
fixed_add = []
fixed_st = []
for i in range(len(df)):
  # print(df.loc[i, 'rLOCATION'], df.loc[i, 'Cross Street'])
  add_split = df.loc[i, 'LOCATION'].split()
  address = " ".join(add_split)
  fixed_add.append(address)
  if df.loc[i, 'Cross Street']=='Missing':
    fixed_st.append(np.nan)
  else:
    st_split = df.loc[i, 'Cross Street'].split()
    st = " ".join(st_split)
    fixed_st.append(st)

# 26, 27

df.insert(26, 'ADDRESS', fixed_add)
df.insert(27, 'CROSS ST', fixed_st)

# LOCATION: Street address of crime incident rounded to the nearest hundred block to maintain anonymity.
print('ADDRESS: ')
print('_________________________________________________________________________')
print(df['ADDRESS'].value_counts(), '\n')

# # Cross Street: Cross Street of rounded Address
print('LOCATION CROSS ST: ')
print('_________________________________________________________________________')
print(df['CROSS ST'].value_counts(), '\n')

# due to data recording, some location fields with missing data are noted as (0°, 0°)
# remove rows where lat or lon is 0
df = df[df['LAT'] != 0]
df = df[df['LON'] != 0]

# # LAT: Latitude of crime
print('LOCATION LATITUDE: ')
print('_________________________________________________________________________')
print(df['LAT'].value_counts(), '\n')

# # LON: Longitude of crime
print('LOCATION LONGITUDE: ')
print('_________________________________________________________________________')
print(df['LON'].value_counts(), '\n')

df['COORDINATES'] = df['LAT'].astype(str) + ', ' + df['LON'].astype(str)
# COORDIANTES: Latitude and longitude of crime
print('LOCATION COORDINATES: ')
print('_________________________________________________________________________')
print(df['COORDINATES'].value_counts(), '\n')

ADDRESS: 
_________________________________________________________________________
6TH ST               443
7TH ST               348
6TH                  292
4100 CRENSHAW BL     281
7TH                  248
                    ... 
10600 BELLAGIO RD      1
16200 JERSEY ST        1
VIA MARINA             1
15800 BRYANT ST        1
9800 GERALD AV         1
Name: ADDRESS, Length: 42149, dtype: int64 

LOCATION CROSS ST: 
_________________________________________________________________________
BROADWAY      572
FIGUEROA      413
WESTERN AV    362
VERMONT AV    317
SAN PEDRO     301
             ... 
GANAHL          1
HOMER           1
86TH            1
WOODLAND        1
AMBROSE ST      1
Name: CROSS ST, Length: 4557, dtype: int64 

LOCATION LATITUDE: 
_________________________________________________________________________
34.2012    1533
34.1016    1455
34.1867    1214
34.1939     884
34.0980     823
           ... 
33.7253       1
34.6828       1
33.7515       1
33.8116       1
33.72

In [ ]:
display(df.head(5))

# double check that added columns are an improvement from previous
print(list(df['DATE OCC'].unique())[:10])
print(list(df['TIME OCC'].unique())[:10])
print(list(df['Date Occd'].unique())[:10])
print(list(df['LOCATION'].unique())[:10])
print(list(df['ADDRESS'].unique())[:10])
print(list(df['Cross Street'].unique())[:10])
print(list(df['CROSS ST'].unique())[:10])
print(list(df['LAT'].unique()[:10]))
print(list(df['LON'].unique()[:10]))
print(list(df['COORDINATES'].unique()[:10]))

# checks out, so we will drop these columns
df.drop(columns=['DATE OCC', 'TIME OCC', 'LOCATION', 'Cross Street', 'LAT', 'LON'], inplace=True)

display(df.head(5))

,DR_NO,Date Rptd,DATE OCC,TIME OCC,Date Occd,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,ADDRESS,CROSS ST,Cross Street,LAT,LON,COORDINATES
0,130508806,2013-04-21,2012-01-01,1,2012-01-01 00:01:00,5,Harbor,522,2,354,THEFT OF IDENTITY,1822 1202,59.0,F,H,501.0,SINGLE FAMILY DWELLING,NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,600 GULF AV,600 GULF AV,NaN,Missing,33.7765,-118.2726,"33.7765, -118.2726"
1,151920788,2015-10-27,2012-01-01,1,2012-01-01 00:01:00,19,Mission,1934,2,354,THEFT OF IDENTITY,1822 1218,24.0,M,B,726.0,POLICE FACILITY,NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,11100 N SEPULVEDA BL,11100 N SEPULVEDA BL,NaN,Missing,34.2721,-118.4675,"34.2721, -118.4675"
2,121706321,2012-02-13,2012-01-01,1200,2012-01-01 12:00:00,17,Devonshire,1798,2,354,THEFT OF IDENTITY,0377 0928 1822,46.0,F,W,501.0,SINGLE FAMILY DWELLING,NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,15800 BAHAMA ST,15800 BAHAMA ST,NaN,Missing,34.2329,-118.4775,"34.2329, -118.4775"
3,170816763,2017-10-10,2012-01-01,800,2012-01-01 08:00:00,8,West LA,889,2,354,THEFT OF IDENTITY,1822,56.0,M,H,502.0,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,2000 S GARTH AV,2000 S GARTH AV,NaN,Missing,34.0408,-118.3791,"34.0408, -118.3791"
4,141215190,2014-06-27,2012-01-01,1200,2012-01-01 12:00:00,12,77th Street,1258,2,813,CHILD ANNOYING (17YRS & UNDER),1251 1259 1820 0319 0522,14.0,F,H,501.0,SINGLE FAMILY DWELLING,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,813.0,NaN,NaN,NaN,73RD ST,73RD ST,MAIN ST,MAIN ST,33.9739,-118.2739,"33.9739, -118.2739"


['2012-01-01', '2012-01-02', '2012-01-03', '2012-01-04', '2012-01-05', '2012-01-06', '2012-01-07', '2012-01-08', '2012-01-09', '2012-01-10']
[1, 1200, 800, 2330, 900, 330, 1900, 1100, 2200, 1730]
[numpy.datetime64('2012-01-01T00:01:00.000000000'), numpy.datetime64('2012-01-01T12:00:00.000000000'), numpy.datetime64('2012-01-01T08:00:00.000000000'), numpy.datetime64('2012-01-01T23:30:00.000000000'), numpy.datetime64('2012-01-01T09:00:00.000000000'), numpy.datetime64('2012-01-01T03:30:00.000000000'), numpy.datetime64('2012-01-01T19:00:00.000000000'), numpy.datetime64('2012-01-01T11:00:00.000000000'), numpy.datetime64('2012-01-01T22:00:00.000000000'), numpy.datetime64('2012-01-01T17:30:00.000000000')]
['600    GULF                         AV', '11100 N  SEPULVEDA                    BL', '15800    BAHAMA                       ST', '2000 S  GARTH                        AV', '73RD                         ST', '8300    KENTLAND                     AV', '700 S  WESTLAKE                     AV',

,DR_NO,Date Rptd,Date Occd,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,ADDRESS,CROSS ST,COORDINATES
0,130508806,2013-04-21,2012-01-01 00:01:00,5,Harbor,522,2,354,THEFT OF IDENTITY,1822 1202,59.0,F,H,501.0,SINGLE FAMILY DWELLING,NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,600 GULF AV,NaN,"33.7765, -118.2726"
1,151920788,2015-10-27,2012-01-01 00:01:00,19,Mission,1934,2,354,THEFT OF IDENTITY,1822 1218,24.0,M,B,726.0,POLICE FACILITY,NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,11100 N SEPULVEDA BL,NaN,"34.2721, -118.4675"
2,121706321,2012-02-13,2012-01-01 12:00:00,17,Devonshire,1798,2,354,THEFT OF IDENTITY,0377 0928 1822,46.0,F,W,501.0,SINGLE FAMILY DWELLING,NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,15800 BAHAMA ST,NaN,"34.2329, -118.4775"
3,170816763,2017-10-10,2012-01-01 08:00:00,8,West LA,889,2,354,THEFT OF IDENTITY,1822,56.0,M,H,502.0,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,2000 S GARTH AV,NaN,"34.0408, -118.3791"
4,141215190,2014-06-27,2012-01-01 12:00:00,12,77th Street,1258,2,813,CHILD ANNOYING (17YRS & UNDER),1251 1259 1820 0319 0522,14.0,F,H,501.0,SINGLE FAMILY DWELLING,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,813.0,NaN,NaN,NaN,73RD ST,MAIN ST,"33.9739, -118.2739"


In [ ]:
df.to_csv('Cleaned_Crime_Data_2012.csv')